# Algorithmic Trading with Predictors

Using the example of crypto pricing from yfinance, we can extract 5-min interval data for, say Ethereum-USD, over a 60 day period. And can similarly overlap with other crypto pairs. The pricing data consists of open, close, high, low and technical-based predicators can be built on that.

The predictor variables are features that might be related to market behavior. These data can be the technical indicators, the market data, the sentiment data, breadth data, fundamental data, government data, etc. 

On the technicals, we can consider simply the close-open delta to determine whether the price has increased or decreased. With the price, % changes, etc. as the features and the target predictor (i.e. whether close is higher than open) as a label, we can train a machine learning model to attempt to forecast this.

In [61]:
!pip install yfinance

In [62]:
from yfinance import Ticker
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

We use yfinance to download the data, pandas for data processing, and sklearn to split the data for training and testing. The split will be an important step here because we need to make sure everything still works out of the sample. XGBoost has been used due to its stellar out-of-box performance and no requirement for feature scaling.

In [76]:
def get_ohlc(ticker, period='60d', interval='5m'):
  cols = ['Open','High','Low','Close']
  ohlc = Ticker(ticker).history(period=period, interval=interval)[cols][:-1]
  return ohlc

df = get_ohlc('ETH-USD')

This function downloads open, high, low, and close prices for any given cryptocurrency pair and timeframe. There are limits, for example, only 60 days are available for the 5-minute interval. That gives us roughly 17,000 rows which is still a decent sample size for quick testing.

In [77]:
df

,Open,High,Low,Close
Datetime,,,,
2022-01-13 00:00:00+00:00,3372.104980,3377.017334,3372.104980,3377.017334
2022-01-13 00:05:00+00:00,3377.922363,3380.488037,3377.875977,3380.488037
2022-01-13 00:10:00+00:00,3381.010986,3381.010986,3377.831543,3377.831543
2022-01-13 00:15:00+00:00,3377.104004,3377.104004,3375.414795,3375.504395
2022-01-13 00:20:00+00:00,3375.253662,3375.253662,3373.264648,3373.385254
...,...,...,...,...
2022-03-13 17:50:00+00:00,2579.877930,2579.877930,2579.262451,2579.454102
2022-03-13 17:55:00+00:00,2579.404785,2579.404785,2579.035889,2579.329346
2022-03-13 18:00:00+00:00,2579.282471,2579.746582,2579.204834,2579.204834


One of the simplest things to predict is whether the price of the next bar will go up or down. Ie. based on previous data, will the closing price of the next bar be higher than the opening price. 

In [78]:
df['target'] = df['Close'] > df['Open']

In [79]:
df['target'] = df['target'].shift(-1)
df.dropna(inplace=True)

In [80]:
df

,Open,High,Low,Close,target
Datetime,,,,,
2022-01-13 00:00:00+00:00,3372.104980,3377.017334,3372.104980,3377.017334,True
2022-01-13 00:05:00+00:00,3377.922363,3380.488037,3377.875977,3380.488037,False
2022-01-13 00:10:00+00:00,3381.010986,3381.010986,3377.831543,3377.831543,False
2022-01-13 00:15:00+00:00,3377.104004,3377.104004,3375.414795,3375.504395,False
2022-01-13 00:20:00+00:00,3375.253662,3375.253662,3373.264648,3373.385254,True
...,...,...,...,...,...
2022-03-13 17:45:00+00:00,2583.430664,2583.430664,2579.864258,2579.864258,False
2022-03-13 17:50:00+00:00,2579.877930,2579.877930,2579.262451,2579.454102,False
2022-03-13 17:55:00+00:00,2579.404785,2579.404785,2579.035889,2579.329346,False


The method pct_change finds the row before to do the percentage change

In [81]:
df['pct_change1'] = df['Close'].pct_change()
df['pct_change2'] = df['Open'].pct_change()

In [82]:
df

,Open,High,Low,Close,target,pct_change1,pct_change2
Datetime,,,,,,,
2022-01-13 00:00:00+00:00,3372.104980,3377.017334,3372.104980,3377.017334,True,NaN,NaN
2022-01-13 00:05:00+00:00,3377.922363,3380.488037,3377.875977,3380.488037,False,0.001028,0.001725
2022-01-13 00:10:00+00:00,3381.010986,3381.010986,3377.831543,3377.831543,False,-0.000786,0.000914
2022-01-13 00:15:00+00:00,3377.104004,3377.104004,3375.414795,3375.504395,False,-0.000689,-0.001156
2022-01-13 00:20:00+00:00,3375.253662,3375.253662,3373.264648,3373.385254,True,-0.000628,-0.000548
...,...,...,...,...,...,...,...
2022-03-13 17:45:00+00:00,2583.430664,2583.430664,2579.864258,2579.864258,False,-0.001480,-0.000159
2022-03-13 17:50:00+00:00,2579.877930,2579.877930,2579.262451,2579.454102,False,-0.000159,-0.001375
2022-03-13 17:55:00+00:00,2579.404785,2579.404785,2579.035889,2579.329346,False,-0.000048,-0.000183


In [83]:
df.drop(df.index[0], inplace=True)

In [84]:
df["target"] = df["target"].astype(int)

In [85]:
X = df.drop(columns='target')
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, test_size=.50, shuffle=False)

In [25]:
model = XGBClassifier()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5305432501748659

# Advanced Predictors

Find rolling averages; use advanced dataframe manipulation tools to find local maxima and minima, then find time taken through index searching of when the average hits a threshold (e.g. 90% of local maxima).

In [86]:
df['Mid'] = (df['High'] + df['Low'])/2

In [87]:
df['rolling10'] = df['Mid'].rolling(10).mean()

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17194 entries, 2022-01-13 00:05:00+00:00 to 2022-03-13 18:05:00+00:00
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Open         17194 non-null  float64
 1   High         17194 non-null  float64
 2   Low          17194 non-null  float64
 3   Close        17194 non-null  float64
 4   target       17194 non-null  int64  
 5   pct_change1  17194 non-null  float64
 6   pct_change2  17194 non-null  float64
 7   Mid          17194 non-null  float64
 8   rolling10    17185 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 1.3 MB


In [89]:
df = df.reset_index()

In [90]:
df['Day'] = df['Datetime'].dt.date

The below filter and groupby combination will first filter the dataframe by a particular column - in this case the pct_change1 column where the interval-on-interval close price shows growth; then the rolling10 price is grouped by the day and cumulative sum is obtained. Not sure whether this is a viable indicator... but shows the power of pandas dataframe.

In [91]:
df['Cumsum_pos_price'] = df[df['pct_change1'] >= 0].groupby(['Day'])['rolling10'].apply(
    lambda x: x.fillna(0).cumsum())

Applying some of the ideas individually, we can obtain the cumulative time for certain thresholds or whatever to be reached. In the following, examine the data grouped by day but could just as easily be grouped by different investments or another characteristic.

In [93]:
df = df.dropna(subset=['rolling10'])

In [95]:
df_subset = df.groupby(['Day'])['rolling10'].idxmax()

In [96]:
df_subset = df_subset.reset_index()
df_subset.columns = ['Day', 'MaxIndex']

In [97]:
df_subset['MaxAmount'] = df_subset['MaxIndex'].apply(lambda x: df['rolling10'].loc[x])

And similarly for the minimum of each day...

In [103]:
df_subset['MinIndex'] = df.groupby(['Day'])['rolling10'].idxmin().tolist()
df_subset['MinAmount'] = df_subset['MinIndex'].apply(lambda x: df['rolling10'].loc[x])

In [106]:
df_subset['MidAmount'] = (df_subset['MinAmount'] + df_subset['MaxAmount']) / 2

The below uses this "MidAmount" pricing for each day, to then search the original dataframe, per day, for the index at which the rolling10 price is closest to the target "MidAmount". This can them be used to find the time between periods of low-mid-high.

In [108]:
df_subset['MidIndex'] = df_subset[['Day', 'MidAmount']].apply(lambda x: abs(df.loc[(df['Day'] ==
                        x['Day']), 'rolling10'] - x['MidAmount']).idxmin(), axis=1).tolist()

In [109]:
df_subset

,Day,MaxIndex,MaxAmount,MinIndex,MinAmount,MidAmount,MidIndex
0,2022-01-13,180,3384.079553,281,3256.149329,3320.114441,190
1,2022-01-14,559,3324.922852,434,3209.715088,3267.318970,334
2,2022-01-15,826,3360.405481,736,3286.612964,3323.509222,715
3,2022-01-16,1069,3372.653809,1004,3308.144116,3340.398962,1039
4,2022-01-17,1151,3355.307678,1421,3166.257275,3260.782477,1298
5,2022-01-18,1458,3230.619104,1665,3102.861572,3166.740338,1583
6,2022-01-19,1757,3167.993213,1831,3060.822473,3114.407843,1873
7,2022-01-20,2215,3252.527429,2302,3025.249646,3138.888538,2168
8,2022-01-21,2303,3020.537915,2585,2553.294934,2786.916425,2520
9,2022-01-22,2607,2602.644934,2832,2344.625061,2473.634998,2750
